<a href="https://colab.research.google.com/github/c820306/Alpha-camp/blob/main/A2_%E5%AF%A6%E4%BD%9C%E3%80%8Crule_based%E3%80%8D%E7%9A%84%E6%8E%A8%E8%96%A6%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 1.匯入資料

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-25 02:05:22--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  8.88MB/s    in 1.7s    

2021-12-25 02:05:24 (8.88 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2021-12-25 02:05:24--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  6.63MB/s    in 1.5s    

202

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head(3)

category tech1  ... imageURL imageURLHighRes
0       []        ...       []              []
1       []        ...       []              []
2       []        ...       []              []

[3 rows x 19 columns]

In [5]:
ratings.head(3)

asin      reviewerID  overall  unixReviewTime
0  0143026860  A1V6B6TNIC10QE      1.0      1424304000
1  0143026860  A2F5GHSXFQ0W6J      4.0      1418860800
2  0143026860  A1572GUYS7DGSR      4.0      1407628800

## 3.資料整理

In [6]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 4.資料切分

In [22]:
ratings_trainings1 = ratings[(ratings['DATE'] < '2018-09-01')]
ratings_trainings2 = ratings[(ratings['DATE'] >= '2018-06-01') & (ratings['DATE'] < '2018-09-01')]
ratings_testings = ratings[(ratings['DATE'] >= '2018-09-01') &  (ratings['DATE'] <= '2018-09-30')]

print("ratings_trainings1:", ratings_trainings1.shape)
print("ratings_trainings2:", ratings_trainings2.shape)
print("ratings_testings:", ratings_testings.shape)

ratings_trainings1: (370752, 5)
ratings_trainings2: (7027, 5)
ratings_testings: (590, 5)


In [23]:
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records') #‘records’ : list like [{column -> value}, … , {column -> value}]

In [24]:
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }

## 4.產生推薦

In [25]:
def recommender(ratings_trainings1, ratings_trainings2, k=15):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * k: int 每個使用者需要推薦的商品數
    * recommendations: list  [推薦商品一, 推薦商品二, ...]
    '''
    
    ratings_compare = ratings_trainings1.groupby(['asin'], as_index=False).agg(product_count=("asin", 'count'),avg_rating=("overall", 'mean'), nearest_date=('DATE','max'))
    
    ## method 1: product_count
    rating_sorting = ratings_compare.sort_values(by=['product_count'], ascending=(False))
    rating_sorting['ranking'] = rating_sorting['product_count'].rank(ascending=False)
    rating_sorting.reset_index(drop=True, inplace=True)
    recom6= rating_sorting['asin'].tolist()[:k]
    
    ## method 2: avg_rating
    rating_sorting = ratings_compare.sort_values(by=['avg_rating'], ascending=(False))
    rating_sorting['ranking'] = rating_sorting['avg_rating'].rank(ascending=False)
    rating_sorting.reset_index(drop=True, inplace=True)
    recom2= rating_sorting['asin'].tolist()[:k]
    
    ## method 3: product_count>avg_rating
    rating_sorting = ratings_compare.sort_values(by=['product_count','avg_rating'], ascending=(False,False))
    rating_sorting['ranking'] = rating_sorting['product_count'].rank(ascending=False)
    rating_sorting.reset_index(drop=True, inplace=True)
    recom3= rating_sorting['asin'].tolist()[:k]
    
    ## method 4: avg_rating>product_count
    rating_sorting = ratings_compare.sort_values(by=['avg_rating','product_count'], ascending=(False,False))
    rating_sorting['ranking'] = rating_sorting['avg_rating'].rank(ascending=False)
    rating_sorting.reset_index(drop=True, inplace=True)
    recom4= rating_sorting['asin'].tolist()[:k]
    
    '''
    * 分析近三個月爆品
    * training_data: dataframe 輸入的訓練資料集（2018-09-01~2018-09-01）
    * k: int 每個使用者需要推薦的商品數
    * recommendations: list  [推薦商品一, 推薦商品二, ...]
    '''
    ratings_compare = ratings_trainings2.groupby(['asin'], as_index=False).agg(product_count=("asin", 'count'),avg_rating=("overall", 'mean'), nearest_date=('DATE','max'))

    ## method 5: recent_popular: product_count
    rating_sorting = ratings_compare.sort_values(by=['product_count'], ascending=(False))
    rating_sorting['ranking'] = rating_sorting['product_count'].rank(ascending=False)
    rating_sorting.reset_index(drop=True, inplace=True)
    recom5= rating_sorting['asin'].tolist()[:k]
    
    ## method 6: recent_popular: avg_rating
    rating_sorting = ratings_compare.sort_values(by=['avg_rating'], ascending=(False))
    rating_sorting['ranking'] = rating_sorting['avg_rating'].rank(ascending=False)
    rating_sorting.reset_index(drop=True, inplace=True)
    recom6= rating_sorting['asin'].tolist()[:k]

    
    return recom1, recom2,recom3,recom4,recom5,recom6

recom1, recom2,recom3,recom4,recom5,recom6 = recommender(ratings_trainings1, ratings_trainings2)


## 5.結果評估

In [41]:
evaluate_group ={"product_count":recom1,
                 "avg_rating":recom2,
                 "product_count> avg_rating":recom3,
                 "avg_rating> product_count":recom4,
                 "product_count(recently 3_months)":recom5,
                 "avg_rating(recently 3_months)":recom6
                }

def evaluate(ratings_testings_by_user, evaluate_group):
    '''
    * 評估各方法前Ｎ大推薦商品，與測試集實際銷售交集數量，即可得出個方法推薦命中率
    * ratings_testings_by_user ： dict  {user1: [asin1,asin2], user2:[asin1], ....}
    * evaluate_group : dict  {method1: recommendation list, method2: recommendation list,...}
    * score: %
    '''
    print("Evaluation result: \n")
    for group, recommendations in evaluate_group.items():
        total = 0
        for d in ratings_testings_by_user.keys():
            total += len(set(evaluate_group[group])& set(ratings_testings_by_user[d]))
        score = total / len(ratings_testings)
        
        print(group,":", round(score*100,2),"%\n")
evaluate(ratings_testings_by_user, evaluate_group)

Evaluation result: 

product_count : 8.31 %

avg_rating : 0.0 %

product_count> avg_rating : 8.31 %

avg_rating> product_count : 0.0 %

product_count(recently 3_months) : 15.42 %

avg_rating(recently 3_months) : 0.17 %

